In [97]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Input
from keras.optimizers import SGD
from keras import backend as K
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras import regularizers
from keras.layers import InputLayer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, accuracy_score
import keras

import pandas as pd
import numpy as np

In [98]:
df = pd.read_csv("content/combined_data_10.csv")

In [99]:
df.columns[:15]

Index(['imdb_title_id', 'title', 'year', 'duration', 'budget',
       'weighted_average_vote', 'height', 'children', 'divorces', 'spouses',
       'actor_age_at_release', 'actor_age', 'genre_Drama', 'genre_Comedy',
       'genre_Comedy, Drama'],
      dtype='object')

In [100]:
y = df["weighted_average_vote"].astype(float)
X = df.drop(
    ["imdb_title_id", "title", "weighted_average_vote"],
    axis=1,
).astype(float)

In [101]:
X.shape

(15690, 88)

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

dim = X.shape[1]

### To Do
- Change learning rate
- Change test train split

## Nolan's NN

In [132]:
model = Sequential()
model.add(
    Dense(dim * 2, input_dim=dim, activation="relu")
)
model.add(Dense(dim, activation="sigmoid"))
model.add(Dense(10, activation="relu"))
model.add(Dense(1, activation="linear"))

In [133]:
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='mae', 
              optimizer=opt,
              metrics=['mae'])

In [135]:
model.fit(X_train, y_train, epochs=10, batch_size=1024)

yhat = model.predict(X_test)
error = mean_absolute_error(y_test, yhat)
print("MAE: %.3f" % error)

Epoch 1/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7668 - mae: 0.7668
Epoch 2/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7638 - mae: 0.7638
Epoch 3/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7619 - mae: 0.7619
Epoch 4/10
10/10 [==============================] - 0s 6ms/step - loss: 0.7590 - mae: 0.7590
Epoch 5/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7580 - mae: 0.7580
Epoch 6/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7571 - mae: 0.7571
Epoch 7/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7563 - mae: 0.7563
Epoch 8/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7555 - mae: 0.7555
Epoch 9/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7549 - mae: 0.7549
Epoch 10/10
10/10 [==============================] - 0s 5ms/step - loss: 0.7546 - mae: 0.7546
MAE: 0.784


## ReLU

In [106]:
X_train.shape

(9414, 88)

In [107]:
model_seq2 = Sequential()
model_seq2.add(Dense(dim, input_dim=dim))
model_seq2.add(Activation('relu'))
model_seq2.add(Dense(dim/2))
model_seq2.add(Activation('relu'))
model_seq2.add(Dense(20))
model_seq2.add(Activation('relu'))
model_seq2.add(Dense(10))
model_seq2.add(Activation('relu'))
model_seq2.add(Dense(1))
model_seq2.add(Activation('linear'))

model_seq2.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 88)                7832      
_________________________________________________________________
activation_6 (Activation)    (None, 88)                0         
_________________________________________________________________
dense_38 (Dense)             (None, 44)                3916      
_________________________________________________________________
activation_7 (Activation)    (None, 44)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 20)                900       
_________________________________________________________________
activation_8 (Activation)    (None, 20)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 10)               

In [108]:
opt = SGD(learning_rate=3.0)

model_seq2.compile(loss='mae', 
              optimizer=opt,
              metrics=['mae'])

In [109]:
model_seq2.fit(X_train, y_train, epochs=10, batch_size=64)

yhat = model.predict(X_test)
error = mean_absolute_error(y_test, yhat)
print("MAE: %.3f" % error)

Epoch 1/10
148/148 [==============================] - 2s 2ms/step - loss: 1314017767874958848.0000 - mae: 1314017767874958848.0000
Epoch 2/10
148/148 [==============================] - 0s 3ms/step - loss: 1.1300 - mae: 1.1300
Epoch 3/10
148/148 [==============================] - 0s 3ms/step - loss: 1.1416 - mae: 1.1416
Epoch 4/10
148/148 [==============================] - 0s 2ms/step - loss: 1.1096 - mae: 1.1096
Epoch 5/10
148/148 [==============================] - 0s 2ms/step - loss: 1.1323 - mae: 1.1323
Epoch 6/10
148/148 [==============================] - 0s 2ms/step - loss: 1.1202 - mae: 1.1202
Epoch 7/10
148/148 [==============================] - 0s 2ms/step - loss: 1.1433 - mae: 1.1433
Epoch 8/10
148/148 [==============================] - 0s 2ms/step - loss: 1.2057 - mae: 1.2057
Epoch 9/10
148/148 [==============================] - 0s 2ms/step - loss: 1.1415 - mae: 1.1415
Epoch 10/10
148/148 [==============================] - 0s 2ms/step - loss: 1.1305 - mae: 1.1305
MAE: 0.751


## Sequential Dropout

In [110]:
dropout = 0.15
regularizers = 0.0001

In [111]:
model_seq3 = Sequential()
model_seq3.add(InputLayer(input_shape=dim))
model_seq3.add(Dense(dim, activation="relu"))
model_seq3.add(Dropout(dropout))
model_seq3.add(
    Dense(dim/2, activation="relu", kernel_regularizer=keras.regularizers.l2(regularizers))
)
model_seq3.add(Dropout(dropout))
model_seq3.add(
    Dense(10, activation="relu", kernel_regularizer=keras.regularizers.l2(regularizers))
)
model_seq3.add(Dropout(dropout))
model_seq3.add(Dense(1, activation="linear"))
model_seq3.add(Activation("softmax"))

In [112]:
opt = SGD(learning_rate=3.0)

model_seq3.compile(loss='mean_absolute_error', 
              optimizer=opt,
              metrics=["mae"])

In [113]:
model_seq3.fit(X_train, y_train, epochs=15, batch_size=2048)

Epoch 1/15
5/5 [==============================] - 2s 7ms/step - loss: 5.0267 - mae: 5.0193
Epoch 2/15
5/5 [==============================] - 0s 9ms/step - loss: 5.0229 - mae: 5.0155
Epoch 3/15
5/5 [==============================] - 0s 8ms/step - loss: 5.0333 - mae: 5.0260
Epoch 4/15
5/5 [==============================] - 0s 9ms/step - loss: 5.0319 - mae: 5.0246
Epoch 5/15
5/5 [==============================] - 0s 8ms/step - loss: 5.0216 - mae: 5.0143
Epoch 6/15
5/5 [==============================] - 0s 18ms/step - loss: 5.0248 - mae: 5.0176
Epoch 7/15
5/5 [==============================] - 0s 8ms/step - loss: 5.0299 - mae: 5.0228
Epoch 8/15
5/5 [==============================] - 0s 8ms/step - loss: 5.0228 - mae: 5.0156
Epoch 9/15
5/5 [==============================] - 0s 9ms/step - loss: 5.0173 - mae: 5.0102
Epoch 10/15
5/5 [==============================] - 0s 9ms/step - loss: 5.0355 - mae: 5.0284
Epoch 11/15
5/5 [==============================] - 0s 7ms/step - loss: 5.0226 - mae: 5.0

In [114]:
yhat = model_seq3.predict(X_test)
error = mean_absolute_error(y_test, yhat)
print("MAE: %.3f" % error)

MAE: 5.035
